In [ ]:
import requests
import json

response = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(response.text)

len(data), print(type(data))

In [ ]:
def json_print(data):
    print(json.dumps(data, indent=2))


json_print(data[0])

In [ ]:
import weaviate, os
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')


In [ ]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions(),  
    additional_headers={
        "X-OpenAI-Api-Key": openai.api_key 
    }
)

print(f"Client created? {client.is_ready()}")

In [ ]:
json_print(client.get_meta())

In [ ]:
if client.schema.exists("Question"):
    client.schema.delete_class("Question")
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  
}

client.schema.create_class(class_obj)

In [ ]:
json_print(data[0])

In [ ]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

In [ ]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

In [ ]:
result=(client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(1)
          .do())

json_print(result)

In [ ]:
result=(client.query
          .get("Question", ["category", "question", "answer"])
          .with_near_text({"concepts": "biology"})
          .with_additional('distance')
          .with_limit(2)
          .do())

json_print(result)

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"],"distance":0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

# Vector Databases support for CRUD operations

## Create

In [ ]:
object_uuid=client.data_object.create(
    data_object={
        'question':"Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"
    },
    class_name="Question"
)

print(object_uuid)

## Read

In [ ]:
data_object=client.data_object.get_by_id(object_uuid)
print(data_object)

## Update

In [ ]:
client.data_object.update(uuid=object_uuid,class_name="Question",data_object={
        'answer':"Florence, Italy"
    })

In [ ]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

json_print(data_object)

## Delete

In [ ]:
json_print(client.query.aggregate("Question").with_meta_count().do())